# Models Evaluation

1. Create models
2. Compare models
3. Export .csv file with results

In [1]:
# Load libraries
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
train_data = pd.read_pickle("train_data.pkl")
test_data = pd.read_pickle("test_data.pkl")

train_data

,loan_id,account_id,amount_loan,duration,payments,status,disp_id,client_id,district_id_clt,age_clt,gender_clt,district_id_acc,frequency,age_acc
0,5314,1787,96396,12,8033,-1,2166,2166,30,74,0,30,2,28
1,5316,1801,165960,36,4610,1,2181,2181,46,53,1,46,1,28
2,6863,9188,127080,60,2118,1,11006,11314,45,85,1,45,1,28
3,5325,1843,105804,36,2939,1,2235,2235,14,81,0,12,1,28
4,7240,11013,274740,60,4579,1,13231,13539,63,43,1,1,2,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,6818,9030,155616,48,3242,1,10813,11121,72,50,0,72,1,26
324,5625,3189,222180,60,3703,-1,3855,3855,29,82,1,29,1,25
325,6805,8972,45024,48,938,1,10742,11050,70,64,0,70,1,25
326,7233,10963,115812,36,3217,1,13172,13480,16,68,1,16,1,26


In [3]:
features = list(train_data.columns)
features.remove('status')
features.remove('loan_id')

x = train_data[features]
y = train_data['status']

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0) 

In [5]:
# Create Decision Tree classifer object
dtc = DecisionTreeClassifier(min_samples_leaf=10)

# Train Decision Tree Classifer
dtc = dtc.fit(x_train, y_train)

#Predict the response for test dataset
y_pred = dtc.predict_proba(x_test)[:, -1]

In [6]:
# Area Under the Curve

auc = metrics.roc_auc_score(y_test, y_pred)
print("AUC Score: ", auc)

AUC Score:  0.607983193277311


In [7]:
x_real = test_data[features]

predictions = dtc.predict_proba(x_real)[:, -1]

submission = pd.DataFrame()
submission['Id'] = test_data['loan_id']
submission['Predicted'] = predictions

submission.to_csv("submission.csv", index=False)

In [8]:
submission

,Id,Predicted
0,5895,1.000000
1,7122,0.714286
2,6173,0.684211
3,6142,0.800000
4,5358,1.000000
...,...,...
349,4989,1.000000
350,5221,0.800000
351,6402,1.000000
352,5346,1.000000
